In [ ]:
import numpy as np 
import pandas as pd
import vrplib
import collections
from scipy.cluster.hierarchy import linkage, fcluster,inconsistent
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# read VRPLIB formatted instances (default)

instance = vrplib.read_instance("C:/Users/cevat/Desktop/assignment/CVRP-Heuristics-Lab/Vrp-Set-D/Vrp-Set-D/D/ORTEC-n242-k12.vrp")
solution = vrplib.read_solution("C:/Users/cevat/Desktop/assignment/CVRP-Heuristics-Lab/Vrp-Set-D/Vrp-Set-D/D/ORTEC-n242-k12.sol")
instance.keys()

In [ ]:
#def solve_cvrp_gurobi(instance,vehicle_count=round(sum(instance["demand"])/ instance["capacity"]*2)):


# Unpack instance information
vehicle_count=round(sum(instance["demand"])/ instance["capacity"]*2)
edge_weight = instance["edge_weight"]
demand = instance["demand"]
node_count = len(demand)
vehicle_capacity = instance["capacity"]

# Create a new Gurobi model
model = gp.Model("CVRP")

# Enable Gurobi logs
model.setParam('OutputFlag', 1)


# Create variables
x = model.addVars(node_count, node_count, vehicle_count, vtype=GRB.BINARY, name="x")
u = model.addVars(node_count, vehicle_count, vtype=GRB.CONTINUOUS, lb=0, name="u")

# Set objective: Minimize the total distance traveled
model.setObjective(gp.quicksum(edge_weight[i][j] * x[i, j, k] 
                                for i in range(node_count) 
                                for j in range(node_count) 
                                for k in range(vehicle_count)), GRB.MINIMIZE)

# Constraints

# Each customer must be visited exactly once by exactly one vehicle
model.addConstrs((gp.quicksum(x[i, j, k] for j in range(node_count) 
                for k in range(vehicle_count) 
                if i != j) == 1 for i in range(1, node_count)), name="visit_once")

# Flow conservation at the depot
model.addConstrs((gp.quicksum(x[0, j, k] for j in range(1, node_count)) == 1 
                for k in range(vehicle_count)), name="leave_depot")
model.addConstrs((gp.quicksum(x[i, 0, k] for i in range(1, node_count)) == 1 
                for k in range(vehicle_count)), name="return_depot")

# Capacity constraints
model.addConstrs((gp.quicksum(demand[i] * x[i, j, k] for i in range(node_count) 
                for j in range(1, node_count)) <= vehicle_capacity 
                for k in range(vehicle_count)), name="capacity")

# Flow balance constraints for each node and vehicle
# model.addConstrs(
#    (gp.quicksum(x[i, j, k] for j in range(node_count) if i != j) == gp.quicksum(x[j, i, k] for j in range(node_count) if i != j) 
#        for i in range(1, node_count) for k in range(vehicle_count)), 
#    name="flow_balance")


# Subtour elimination constraints (MTZ constraints)
model.addConstrs((u[i, k] - u[j, k] + node_count * x[i, j, k] <= node_count - 1 for i in range(1, node_count) for j in range(1, node_count) for k in range(vehicle_count) if i != j), name="subtour_elimination")

# Set bounds for u variables (for MTZ subtour elimination)
model.addConstrs((u[i, k] >= demand[i] for i in range(1, node_count) for k in range(vehicle_count)), name="u_lower_bound")
model.addConstrs((u[i, k] <= vehicle_capacity for i in range(1, node_count) for k in range(vehicle_count)), name="u_upper_bound")



# Optimize the model
model.optimize()

# Extract the solution
if model.status == GRB.OPTIMAL:
    solution = {}
    for k in range(vehicle_count):
        route = []
        current_node = 0  # Start from the depot
        while True:
            next_node = None
            for j in range(node_count):
                if j != current_node and x[current_node, j, k].x > 0.5:
                    route.append(j)
                    next_node = j
                    break
            if next_node is None or next_node == 0:
                break
            current_node = next_node
        solution[k] = route
#         return solution
else:
    print("No optimal solution found")
#        return None
